In [1]:
# implementing lenet - 5 layers
# based on https://blog.paperspace.com/writing-lenet5-from-scratch-in-python/

![Alt text](https://blog.paperspace.com/content/images/size/w1000/2021/10/image-17.png)

In [2]:
# imports 
import torch 
import torch.nn as nn 
import torchvision 
import torchvision.transforms as transforms
import numpy as np

batch_size = 64 
num_classes = 10 
learning_rate = 0.001
num_epochs = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'mps')


In [3]:
# dataset
transform = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor(), transforms.Normalize(mean= (0.1325,), std=(0.3105,))])

train_dataset = torchvision.datasets.MNIST(root='alexnet/data', train=True,transform=transform,download=True)
test_dataset = torchvision.datasets.MNIST(root='alexnet/data', train=False,transform=transform,download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size= batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size= batch_size, shuffle=True)

In [7]:
# model 
class LeNet5(nn.Module):
    def __init__(self, num_classes): 
        super(LeNet5, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2)
        ) # meaning sequence of operation
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.fc = nn.Linear(400,120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        # reshape
        out = out.reshape(out.size(0),-1)
        
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        
        return out
        
        
        
        

In [8]:
# hyperparameters
model = LeNet5(num_classes=10).to(device)

# loss function 
cost = nn.CrossEntropyLoss()

# optimizer 
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)


In [9]:
# training
total_step = len(train_loader)

for epoch in range(num_classes): 
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # forward 
        outputs = model(images)
        loss = cost(outputs, labels)
        
        # backward 
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        
        if (i+1) % 400 == 0: 
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [400/938], Loss: 0.0899
Epoch [1/10], Step [800/938], Loss: 0.0403
Epoch [2/10], Step [400/938], Loss: 0.0836
Epoch [2/10], Step [800/938], Loss: 0.0935
Epoch [3/10], Step [400/938], Loss: 0.0022
Epoch [3/10], Step [800/938], Loss: 0.0143
Epoch [4/10], Step [400/938], Loss: 0.1244
Epoch [4/10], Step [800/938], Loss: 0.0155
Epoch [5/10], Step [400/938], Loss: 0.0091
Epoch [5/10], Step [800/938], Loss: 0.0219
Epoch [6/10], Step [400/938], Loss: 0.0045
Epoch [6/10], Step [800/938], Loss: 0.0059
Epoch [7/10], Step [400/938], Loss: 0.0026
Epoch [7/10], Step [800/938], Loss: 0.0306
Epoch [8/10], Step [400/938], Loss: 0.0070
Epoch [8/10], Step [800/938], Loss: 0.0171
Epoch [9/10], Step [400/938], Loss: 0.0036
Epoch [9/10], Step [800/938], Loss: 0.0025
Epoch [10/10], Step [400/938], Loss: 0.0060
Epoch [10/10], Step [800/938], Loss: 0.0833


In [11]:
# testing 
with torch.no_grad(): 
    correct = 0 
    total = 0 
    
    for images,labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data,1)
        total+=labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 99.22 %
